In [1]:
%pip install gspread pandas google-auth

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gspread
from google.oauth2 import service_account as sa
import pandas as pd
from datetime import datetime


scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

#Establezco los alcances de la API de Google Sheets y Google Drive para poder obtener acceso
# a la hoja de cálculo de Google Sheets.
KEY = 'key.json'
#Llave de autenticación de Google Cloud Platform que sera proporcionada mediante zip
#ya que me revoca los permisos si google nota se hace publico el archivo

ID = '1p8l7NPbFjC9QV9WkPpR4LrRpem8v--DmimETEQbyS-M'
#ID de la hoja de cálculo de Google Sheets con link
#https://docs.google.com/spreadsheets/d/1p8l7NPbFjC9QV9WkPpR4LrRpem8v--DmimETEQbyS-M/edit?gid=0#gid=0


credenciales  = sa.Credentials.from_service_account_file(KEY, scopes=scopes)

cliente = gspread.authorize(credenciales)

hoja = cliente.open_by_key(ID).sheet1
#Abro la hoja de cálculo de Google Sheets con el ID proporcionado y selecciono la primera hoja
#previamente creada con datos fictisios de empleados



df = pd.DataFrame(hoja.get_all_records())
print(df)
#visualizo la hoja de cálculo en un DataFrame de Pandas

      Nombre  Salario Mensual Fecha de Contratación
0       Juan            17151            2019-08-02
1        Ana            35945            2019-04-21
2      Pedro            37109            2024-05-31
3      Luisa            16776            2019-06-08
4     Carlos            49393            2016-04-05
5      Marta            43474            2020-05-30
6     Javier            27722            2015-10-25
7      Sofia            34916            2017-04-13
8      Diego            46925            2020-11-04
9      Laura            18352            2019-04-24
10   Alberto            15205            2017-09-16
11    Carmen            20096            2017-04-06
12    Daniel            29359            2019-03-30
13     Maria            26961            2024-07-20
14  Fernando            30177            2019-08-17
15  Patricia            27400            2021-08-26
16      Raul            46996            2014-11-28
17  Gabriela            36593            2018-11-03
18      Luis

In [3]:
df['Fecha de Contratación'] = pd.to_datetime(df['Fecha de Contratación'])
today = datetime.now().date()
#Convierto la columna de Fecha de Contratación a un objeto de fecha y obtengo la fecha actual
#para poder realizar operaciones con fechas

df['Años en la Empresa'] = df['Fecha de Contratación'].apply(lambda x: today.year - x.year)

In [4]:
df['Salario Anual'] = df['Salario Mensual'] * 12
reporte = df[['Nombre', 'Salario Mensual', 'Salario Anual', 'Años en la Empresa']]
print(reporte)
#defino las columnas a filtrar y visualizo el reporte de salarios anuales de los empleados


      Nombre  Salario Mensual  Salario Anual  Años en la Empresa
0       Juan            17151         205812                   5
1        Ana            35945         431340                   5
2      Pedro            37109         445308                   0
3      Luisa            16776         201312                   5
4     Carlos            49393         592716                   8
5      Marta            43474         521688                   4
6     Javier            27722         332664                   9
7      Sofia            34916         418992                   7
8      Diego            46925         563100                   4
9      Laura            18352         220224                   5
10   Alberto            15205         182460                   7
11    Carmen            20096         241152                   7
12    Daniel            29359         352308                   5
13     Maria            26961         323532                   0
14  Fernando            3

In [5]:
reporte_salarios = cliente.open_by_key(ID).add_worksheet('Reporte Salarios', rows=len(reporte), cols=len(reporte.columns))
reporte_salarios.update([reporte.columns.values.tolist()] + reporte.values.tolist())
#actualizo la hoja de cálculo con el reporte de salarios anuales de los empleados

{'spreadsheetId': '1p8l7NPbFjC9QV9WkPpR4LrRpem8v--DmimETEQbyS-M',
 'updatedRange': "'Reporte Salarios'!A1:D21",
 'updatedRows': 21,
 'updatedColumns': 4,
 'updatedCells': 84}